# **YOLOv5 Simplified: Great-Barrier-Reef Train (Adapted by Orkatz, Awsaf, and Cèsar Olivares)** 
![](https://storage.googleapis.com/kaggle-competitions/kaggle/31703/logos/header.png?t=2021-10-29-00-30-04)

## Introduction
Now we have completed finding the COTS in the EDA version, let's take things seriously. This time, we are going to detect COTS with YOLOv5. But first, we have to train detecting COTS with YOLOv5, and then submit predictions using inference over YOLOv5, which I'll demonstrate on the next notebook.

## Install Libraries
The first part is different. You need to install libaries with bash commands. First of all, we need to install WandB via pip. Next, add the apt repository, the ppa:ubuntu-toolchain-r/test. Lastly, update and upgrade libstdc++6. The reason why we have to install all four libraries because these four libraries are the most important keys to train YOLOv5.

In [ ]:
!pip install -qU wandb
!add-apt-repository ppa:ubuntu-toolchain-r/test -y
!apt-get update
!apt-get upgrade libstdc++6 -y

## Import Libraries
After downloading libraries with the bash command lines, we are going to import libraries with Python. In order to train YOLOv5, we need to import groupby from itertools, np from numpy (for linear algebra), tqdm from tqdm.notebook (since we need to setup the pandas function to tqdm), pd from pandas (for CSV processing), os, pickle, cv2 (aka OpenCV), Pool from multiprocessing, ast and glob. We also need to import shutil and sys since we will append the path of the competition dataset, along with Parellel, delayed from joblib, and display, HTML from IPython.display, and lastly but mostly important, the animation and rc from the popular plotting matplotlib module, although we have to set up the rc function with animation and jshtml. We even install and import imagesize too.

In [ ]:
from itertools import groupby
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd
import os
import pickle
import cv2
from multiprocessing import Pool
import ast
import glob

import shutil
import sys
sys.path.append('../input/tensorflow-great-barrier-reef')

from joblib import Parallel, delayed
from IPython.display import display, HTML
from matplotlib import animation, rc
rc('animation', html='jshtml')

## Some Key-Points About Training YOLOv5
Here are the key-points about what to train Yolov5:
* One have to submit prediction(s) using the provided **python time-series API**, meaning that this makes this competition very different from previous Object Detection Competitions, like the global wheat detection.
* Every prediction row needs to include all of the bounding boxes for the image. The submission format seems also **COCO (Common Objects in Context dataset)**, which formats in `[x_min, y_min, width, height]`. 
* The competition metric, F2, tolerates some of the false positives (aka FP) in order to ensure that very few starfish weren't detected. In conclusion, the **false negatives (aka FN)** is way more useful than false positives (aka FP).

### **If you were satisfied and helpful about the "YOLOv5 Simplified: Train" notebook, upvote it since that will support me.**

## WandB
![](https://camo.githubusercontent.com/dd842f7b0be57140e68b2ab9cb007992acd131c48284eaf6b1aca758bfea358b/68747470733a2f2f692e696d6775722e636f6d2f52557469567a482e706e67)

If you haven't heard about the first line of the code cell of the **Install Libraries** section, then you didn't know that this is Weights & Biases. Weights & Biases (aka W&B) is a MLOps platform known for tracking our experiments on everything in computer training. We can use W&B to build a lot of better models faster experiment tracking, dataset vision, and bug-fix discussion, and even demonstrating progress towards milestones. There are some of the awesome features speaking of W&B:
* Known for tracking, comparing, and visualizing ML experiments
* Known for obtaining live metrics, terminal logs, and system stats that were streamed to the centralized dashboard
* Known to explain how their model works, which meant showing graphs of how model versions improved, discussion of bug-fixing, and demonstating progress towards milestones.

More info:
https://docs.wandb.ai/

## The Meta Data
Here are the meta data in this notebook:
* `train_images/` - The folder that contains training set of photos of the form at this format of `video_{video_id}/{video_frame}.jpg`.
* `[train/test].csv` - This is known of the the metadata for all of the images. As along with the other test files, most of the test metadata data is only available to their notebook upon submission over predictions. Only the first few rows are available to download.
* `video_id` - This specifies the ID number of the video image that was part of. 
* `video_frame` - This specifies the frame number of the image within the video. It is expected to see occasional gaps in the frame number from when the diver surfaced.
* `sequence` - This is an ID of a gap-free subset of a given video. All of the sequence ids are not meaningfully ordered.
* `sequence_frame` - This specifies the frame number within a given sequence.
* `image_id` - This specifies an ID code for the images, within a format of `{video_id}-{video_frame}`.
* `annotations` - And finally, this is what we are talking about. These are the bounding boxes of any starfish detections in a string format that can be evaluated directly with Python. It doesn't use the same format as the predictions you will submit thus not available in test.csv. A bounding box is described by the pixel coordinate as `(x_min, y_min)` of its lower left corner within the image together with the `width` and `height` in pixels (This is in COCO format).

After the metadata mentioned in this notebook in a list, let's set up the variable needed for Yolov5 training! First of all, we have to set the fold variable to 4, which fold to train, the remove_nobbox to true because we have to remove images that contains none of the bboxes, the root directory to the competition dataset, the image and label directory variables. Notice that these are the directories to save images and labels.

In [ ]:
FOLD = 4 # this meant that the fold to train or maybe crease...
REMOVE_NOBBOX = True # this meant to exterminate images that has no bbox.
ROOT_DIR = '/kaggle/input/tensorflow-great-barrier-reef'
IMAGE_DIR = '/kaggle/images' # this is the directory to save images
LABEL_DIR = '/kaggle/labels' # this is the directory to save labels

## Create Directories
After creating the variables, let's now create directories in order to train YOLOv5. All we have to do is to run the mkdir command in bash, adding the image and label dir variables, so that way, we can use these directories to save images and labels seperately.

In [ ]:
!mkdir -p {IMAGE_DIR}
!mkdir -p {LABEL_DIR}

## Obtain Paths
After creating the image and label directories, we are now obtaining paths to the images. First of all, we are going to define the get_path function along with the row variable inside. Inside the get_path function, the row with the key, old_image_path, defines the path to the root_dir images. On the other hand, the row variable that has the keys, image/label path, defines the path to the image/label images and txt files. And alas, we finally return the row function.

In [ ]:
def get_path(row):
    row['old_image_path'] = f'{ROOT_DIR}/train_images/video_{row.video_id}/{row.video_frame}.jpg'
    row['image_path'] = f'{IMAGE_DIR}/video_{row.video_id}_{row.video_frame}.jpg'
    row['label_path'] = f'{LABEL_DIR}/video_{row.video_id}_{row.video_frame}.txt'
    return row

Now after defining the get_path function, lets train some train data! First of all, create a variable called df, and set it to read csv of train.csv from the root directory, then set the df variable (or a DataFrame) again to itself to apply progress containing the get_path function in order to train it. Next, the df variable (DataFrame) with the annotations key can be setup'd as a variable to the df variable (DataFrame) with annotations key again but applying progress containing a lambda, x, to ast.literal_eval function with the lambda x inside it to train it again. Finally, display the first two rows of df using the head function being set up with 2.

In [ ]:
# Here's the train data!
df = pd.read_csv(f'{ROOT_DIR}/train.csv')
df = df.progress_apply(get_path, axis=1)
df['annotations'] = df['annotations'].progress_apply(lambda x: ast.literal_eval(x))
display(df.head(2))

After running the train data code cell, you may see that the progress bar for the df variables (DataFrames) and and the df variable (DataFrames) with the annotations key had the estimated progress running and after that, displays something that is similar to train.csv but with the old_image_path, image_path, and label_path keys.

## Counting the Number of BBoxes
In order to check the number of bboxes, we setup the df variable (DataFrames) with the df variable with the annotations key and again, applying the progress containing the get_path function with lambda x to find the len of it. Next, we will define another variable, data and set it to the df variable (DataFrames) with the attribute num_bbox greater than 0 with value counts containing normalize to True times 100. 

In [ ]:
df['num_bbox'] = df['annotations'].progress_apply(lambda x: len(x))
data = (df.num_bbox>0).value_counts(normalize=True)*100
print(f"Number of None BBoxes: {data[0]:0.2f}% | Number of BBoxes: {data[1]:0.2f}%")

As we observed, about 79% (or 80%) of images contained no bboxes. But on the other hand, about 20% (or approximately 21%) of images contained bboxes. And we see that the competition dataset does have mostly lots of images that contained none of the bounding boxes.

## Data Cleanup
In this simplified notebook, we use just **bboxed-images** only (`~5k`). Thus, we can use all of the `~23K` images for training but most images don't have any labels. In conclusion, it would be easier to carry out experiments with **bboxed images** only. So, in order to clean data up, we will make an if statement about the REMOVE_NOBBOX type, then the df variable (DataFrame) would be assigned to the df variable (DataFrame) with the query function over the num_bbox greater than 0 stringed inside.

In [ ]:
if REMOVE_NOBBOX:
    df = df.query("num_bbox>0")

## Writing Images
In order to write images, we need to copy all of the images to the Current Directory (which is `/kaggle/working`) as it doesn't have **access to write**, which is why it is needed for **YOLOv5**. But at least we can make this process go faster by using **Joblib**, because it used **Parallel** computing. To start writing images, we first define a function called make_a_copy with a variable, path, inside. Then, the variable, data is assigned to the other variable, path with the function, split over the strings of '/'. Next, we have to specify filename variable by getting the fragment of data with -1, which the same goes to video_id, but with -2. Then, define new_path into joining path with IMAGE_DIR variable and the format of `{video_id}_{filename}'.` Lastly, use shutil and copy it to the new_path variable from the path variable, although this return statement remained blank...

In [ ]:
def make_a_copy(path):
    data = path.split('/')
    filename = data[-1]
    video_id = data[-2]
    new_path = os.path.join(IMAGE_DIR, f'{video_id}_{filename}')
    shutil.copy(path, new_path)
    return # what?!

After running the make_a_copy function, we assign a variable called image_paths to the df variable (DataFrame) with the old_image_path attribute followed by a tolist function to transform the df variable (DataFrame) to a list. Lastly, assign the underscore character to the Parallel function, which contains the n_jobs being set to -1, and threading being set in backend attribute, thus having another parentheses containing the function delayed that holds the function call, make_a_copy along with the inner parentheses with the path variable included, since it was defined in the for loop over the tqdm function containing the variable, image_paths inside. 

In [ ]:
image_paths = df.old_image_path.tolist()
_ = Parallel(n_jobs=-1, backend='threading')(delayed(make_a_copy)(path) for path in tqdm(image_paths))

## Helper Part for Creating a BBox.
If you didn't know what that part is, then you will see that this is the supporters to create a bounding boxes to detect starfish. But note that this code cell remained hidden, which meant that you'll have to reveal the code cell. So, here's how this code cell work in a code cell (see my comments!).

In [ ]:
# First of all, define a function called voc2yolo containing the variables, image_height, image_width, and bboxes.
def voc2yolo(image_height, image_width, bboxes):
    """
    So, here's how it broke down:
    voc  => [x1, y1, x2, y1]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    
    # Inside of the voc2yolo function, we first setup a variable called bboxes and set it to the inner voc2yolo variable bboxes with the copy function and the astype function containing the float type.
    bboxes = bboxes.copy().astype(float) # if we didn't setup the astype as float then all of the values will be 0 as the voc_pascal dtype is now np.int.
    
    # Next, assign the bboxes with the slices of ... and the other slices from 0 to 2 to the same of them divided by image_width. This applies to the image_height line, but the inner slices is now 1, 3.
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]] / image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]] / image_height
    
    # Then specify the two variables, w and h, to the bboxes variable with the slices of ... and 2 (and 0 for h) minus the bboxes variable with slices of ... and 0 (and 1 for h).
    w = bboxes[..., 2] - bboxes[..., 0]
    h = bboxes[..., 3] - bboxes[..., 1]
    
    # Now, assign bboxes with slices of ... and 0 (and 1) to the variable bboxes with slices of ... and 0 (and 1) plus image_width (and image_height). Thus, assigning bboxes with slices ... and 2 (and 3) to the variables w (and h).
    bboxes[..., 0] = bboxes[..., 0] + w/2
    bboxes[..., 1] = bboxes[..., 1] + h/2
    bboxes[..., 2] = w
    bboxes[..., 3] = h
    
    # Finally, return the bboxes variable.
    return bboxes

# Next, we then create another function called yolo2voc containing the same variables as the voc2yolo function.
def yolo2voc(image_height, image_width, bboxes):
    """
    Again, here's how it broke down:
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]
    """
    
    # Like the voc2yolo function, we again setup the variable called bboxes and assign it to themself with the copy function along with the astype function containing the float type.
    bboxes = bboxes.copy().astype(float) # again, if we didn't setup the astype as float then tall of the values will be 0 as the voc_pascal dtype is now np.int.
    
    # This time, assign the bboxes that has the slices of ... and another inner slices of 0, 2 (1, 3) to itself times the image_width (image_height) variable
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]] * image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]] * image_height
    
    # And now, assign the bboxes variable with the slices of ... and another inner slices of 0, 1 (and 2, 3) to itself (from the first one for the bboxes variable that has the inner slices of 2 and 3) plus (or minus) the variable with the slices of ... and another slices of 2 and 3 (divided by 2 for the bboxes with the slices of ... and another inner slices of 0, 1).
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]] / 2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    
    # Again finally, return the bboxes variable.
    return bboxes

# Finally, create a new function that converts yolo to coco by naming it yolo2coco, containing the variables image_height, image_width, and bboxes.
def yolo2coco(image_height, image_width, bboxes):
    """
    Finally, here's how it broke down (but this time, it's coco):
    yolo (again for the last time) => [xmid, ymid, w, h] (normalized)
    coco (for the debut)           => [xmin, ymin, w, h]
    """
    
    # Again for the last time, we setup the variable called bboxes and assign it to themself with the copy function along with the astype function containing the float type.
    bboxes = bboxes.copy().astype(float) # For the last time, if we didn't setup the astype as float then tall of the values will be 0 as the voc_pascal dtype is now np.int.
    
    # Now, let's denormalize! We assign the bboxes with the slices: ... and another inner slices of zero, two (or 1, 3) to itself times image_width (and image_height).
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]] * image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]] * image_height
    
    # Next, we are now converting the (xmid, ymid) to (xmin, y_min) by assigning the bboxes variable with the slices of ... and the inner slices [0, 1] to itself minus the bboxes variable with the slices of ... and the inner slices, [2, 3] divided by 2.
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]] / 2
    
    # Finally for the last time, return the bboxes variable
    return bboxes

# But we need one more function, we are creating a new function that converts coco to yolo (coco2yolo) with the same three variables just like the yolo2coco function.
def coco2yolo(image_height, image_width, bboxes):
    """
    Here's how it break down for the very last time (this to the yolo...):
    yolo => [xmid, ymid, w, h] (again pretty normalized)
    """
    
    # We setup the variable bboxes just like other three functions to itself to copy with (the copy function) followed by an astype function for a specific dtype of a float.
    bboxes = bboxes.copy().astype(float) # Else all value will be 0 as voc_pascal dtype is np.int.
    
    # Now, lets normalize for the last time! We assign the bboxes with the slices: ... and another inner slices of zero, two (or 1, 3) to itself divided by the image_width (and image_height).
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]] / image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]] / image_height
    
    # Next, we are now converting the (xmid, ymid) to (xmin, y_min) by assigning the bboxes variable with the slices of ... and the inner slices [0, 1] to itself minus the bboxes variable with the slices of ... and the inner slices, [2, 3] divided by 2.
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]] / 2
    
    # Finally for the very last time, return the bboxes variable
    return bboxes
    
# Now, let's create the load_image function (along with the image_path variable inside)!
def load_image(image_path):
    # This function contains one line of code: returning a cv2 module along with the cvtColor function containing cv2 module with the imread function with image_path variable inside and the cv2 module with the COLOR_BGR2RGB attribute function.
    return cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)

# And now, we are now plotting one box (or more) by making a function called plot_one_box. Inside of this function contains 2 variables and 3 hybrids (both variables and attributes): x, img, color being set to None, label set to None, and the line_thickness being set to None.
def plot_one_box(x, img, color=None, label=None, line_thickness=None):
    # The first line of code plots one bounding box on image img. To do so, we are assigning tl to the line_thickness hybrid (variable or attribute) comparison to the round function containing the equation of 0.002 times the parentheses of img variable with the shape index of 0 plus the img variable with the shape index of 1 and the plus 1 outside of the round function.
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1])) + 1 # Thus, this represents the line/font thickness.
    # Then, we setup a variable called color and assign it to another variable/attribute that also named color comparing it random module with the randint function containing the numbers 0 and 255 followed by a for loop containing the underscore variable in the range function of 3.
    color = color or [random.randint(0, 255) for _ in range(3)]
    # Next, we created two variables, c1 and c2, and assign them to each 'x' variables with indexes from 0 to 3 that is each locked inside the int function, which is sealed inside the two parentheses.
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    # Now, we create a rectangle by using the cv2 module with the rectangle function containing the 4 variables: img, c1, c2, and color, thus the 2 attributes: thickness being set to the tl variable and the lineType being set to the cv2 module with the LINE_AA attribute.
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    
    # We then make an if statement saying that if there's an label variable, then...
    if label:
        # the variable called tf is assigned to the max function containing two items the tl variable minus 1, and 1.
        tf = max(tl - 1, 1) # Note: This is for font thickness
        # the variable called t_size is assigned to the cv2 module with the getTextSize function containing 4 items: the label variable, a 0, a fontScale attribute being set to the tl variable divided by 3, and the thickness attribute being set to the tf variable along with the index of 0 on the outside of the getTextSize function.
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        # the c2 variable is assigned to the c1 variable with the slice index of 0 plus the t_size variable with 0 along with the c1 variable with the slice index of 1 minus the t_size variable with a slice index of 1 minus 3.
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        # the rectangle shape is filled when the cv2 module with the rectangle function that contains 6 objects: the variables img, c1, c2, color, the number -1, and the cv2 module with the LINE_AA attribute.
        cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)
        # the cv2 module with the putText function is being setup along with the inner objects: the variables img and label, c1 with the slice index of 0 and c1 with the slice index of 1 minus 2 inside parentheses, the number 0, the tl variable divided by 3, a list of four 225s, the thickness attribute setup'd to the tf variable, and finally, the lineType attribute setup'd to the cv2 module with the LINE_AA attribute.
        cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 225, 225, 225], thickness=tf, lineType=cv2.LINE_AA)
        
# Now to the fun part, let's draw bounding boxes! In order to do so, we will create a function called draw_bboxes, containing the variables img, bboxes, classes, class_ids, and the attributes colors being set to None, show_classes being set to None, bbox_format being set to 'yolo', class_name set to False, and line_thickness being set to 2.
def draw_bboxes(img, bboxes, classes, class_ids, colors=None, show_classes=None, bbox_format='yolo', class_name=False, line_thickness=2):
    # First of all, we define a variable called image and assign it to the variable that is inside the function, img with the copy function.
    image = img.copy()
    # Then, define a variable called show_classes which we will assign it to the classes variable inside the draw_bboxes function along with an if statement showing that if the variable show_classes is set to None, thus otherwise the show_classes variable is being set.
    show_classes = classes if show_classes is None else show_classes
    # Now, we will define a variable called colors and we will assign it to the green (or any) color (you'll like) of the RGB format if the colors variable inside the draw_bboxes function is set to None otherwise the colors variable inside the draw_bboxes function is declared.
    colors = (0, 255, 0) if colors is None else colors
    
    # Now, let's classify what type of bbox format is valid for object detection inside of the bbox_format variable! If the bbox_format is equal to yolo, then...
    if bbox_format == 'yolo':
        # the for loop is looping the created idx variable to the len function of the bboxes variable nested by the range function.
        for idx in range(len(bboxes)):
            # Inside the for loop, the bbox, cls, cls_id, color variables were defined and assigned to bboxes (or classes, class_ids) variable with the slice index of the idx variable (for the bbox, cls, and cls_id variables) and the colors variable with the slice index of the cls_id variable if the colors variable encased inside the type function is a list otherwise the colors variable is defined.
            bbox = bboxes[idx]
            cls  = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors
            # Another if statement is called whether the cls variable is in the show_classes variable then...
            if cls in show_classes:
                # the variables xl, y1, w, and h was created and defined to the float function containing the bbox variable with the slice index from 0 to 3 times the image variable with the shape attribute with the slice index of 0 and 1 (divided by 2 for the variables w and h) thus caked inside the round function.
                x1 = round(float(bbox[0]) * image.shape[1])
                y1 = round(float(bbox[1]) * image.shape[0])
                w  = round(float(bbox[2]) * image.shape[1] / 2) # AKA w/2
                h  = round(float(bbox[3]) * image.shape[0] / 2)
                
                # the variable called voc_bbox is defined and assigned to a tuple of the 4 items: the x1 variable minus the w variable, the y1 variable minus the h variable, the x1 variable plus the w variable, and the y1 variable minus the h variable.
                voc_bbox = (x1-w, y1-h, x1+w, y1+h)
                # the plot_one_box was called out with two variables: voc_bbox and image thus the 3 attributes: color set to the color variable, label set to the cls variable if class_name variable is present otherwise the get_label function with the cls is called along with the str caked in, and line_thickness set to the line_thickness variable.
                plot_one_box(voc_bbox, image, color=color, label=cls if class_name else str(cls_id), line_thickness=line_thickness)
                
    # If the bbox_format is not yolo but is coco, then...
    elif bbox_format == 'coco':
        # another for loop is looping the idx variable through the len function holding the bboxes variable inside the range function.
        for idx in range(len(bboxes)):
            # Inside the for loop, the bbox, cls, cls_id, color variables were defined and assigned to bboxes (or classes, class_ids) variable with the slice index of the idx variable (for the bbox, cls, and cls_id variables) and the colors variable with the slice index of the cls_id variable if the colors variable encased inside the type function is a list otherwise the colors variable is defined.
            bbox = bboxes[idx]
            cls  = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors
            
            # Another if statement has appeared whether the cls variable is inside the show_classes variable. If this happen then...
            if cls in show_classes:
                # the variables x1, y1, w, and h is defined just like the yolo part in this code cell. This time, they were assigned to the round function holding the bbox variable with the slice index from 0 to 3 that is caked in the int function.
                x1 = int(round(bbox[0]))
                y1 = int(round(bbox[1]))
                w  = int(round(bbox[2]))
                h  = int(round(bbox[3]))
                
                # the variable, voc_bbox is assigned to a tuple consisting of two variables: x1, y1 and two same variables plus the w (and h) variables.
                voc_bbox = (x1, y1, x1 + w, y1 + h)
                # the plot_one_box is called out with two variables: voc_bbox and image thus the 3 attributes: color set to the color variable, label set to the cls variable if class_name variable is present otherwise the get_label function with the cls is called along with the str caked in, and line_thickness set to the line_thickness variable.
                plot_one_box(voc_bbox, image, color=color, label=cls if class_name else str(cls_id), line_thickness=line_thickness)
                
    # If the bbox_format is not yolo and coco but is voc_pascal then...
    elif bbox_format == 'voc_pascal':
        # another for loop is made looping over idx in the len function along with the range function.
        for idx in range(len(bboxes)):
            # In the for loop, the bbox, cls, cls_id, color variables were defined and assigned to bboxes (or classes, class_ids) variable with the slice index of the idx variable (for the bbox, cls, and cls_id variables) and the colors variable with the slice index of the cls_id variable if the colors variable encased inside the type function is a list otherwise the colors variable is defined just like the yolo and coco part in the code cell.
            bbox = bboxes[idx]
            cls  = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors
            
            # Another if statement is created whether the cls variable is in the show_classes variable then...
            if cls in show_classes:
                # the variables x1 and y1 is defined but this time its different from yolo and coco. Two new variables called x2 and y2 are defined. But like coco, they were assigned to the round function holding the bbox variable with the slice index from 0 to 3 that is caked in the int function.
                x1 = int(round(bbox[0]))
                y1 = int(round(bbox[1]))
                x2 = int(round(bbox[2]))
                y2 = int(round(bbox[3]))
                
                # the voc_bbox variable assigns to a tuple of four variables: x1, y1, x2, y2.
                voc_bbox = (x1, y1, x2, y2)
                # the plot_one_box is called out with two variables: voc_bbox and image thus the 3 attributes: color set to the color variable, label set to the cls variable if class_name variable is present otherwise the get_label function with the cls is called along with the str caked in, and line_thickness set to the line_thickness variable.
                plot_one_box(voc_bbox, image, color=color, label=cls if class_name else str(cls_id), line_thickness=line_thickness)
    
    # If the bbox_format is not all 3 mentioned then...
    else:
        # We will raise the ValueError Exception, alerting that the bbox_format variable has wrong bbox format.
        raise ValueError('Sorry. Wrong BBOX format.')
        
    # Finally, return the image variable
    return image

# Let's obtain the BBoxes! we define a function called get_bbox with annots variable inside.
def get_bbox(annots):
    # The bboxes variable is defined and assigned to a list function containing the annot variable with the values function, which contained a for loop outside of the list function of the annot variable in the annots variable inside this function surrounded by the brackets.
    bboxes = [list(annot.values()) for annot in annots]
    # Finally, return the bboxes variable.
    return bboxes

# We might see the image size, but how do we get it? Well, we will create a function called get_imgsize with a variable inside called row.
def get_imgsize(row):
    # The row variables that is inside the get_imgsize function with two keys, 'width' and 'height' is assigned to the imagesize module linked with the get function that contains the row function with the 'image_path' key.
    row['width'], row['height'] = imagesize.get(row['image_path'])  # We don't need to import imagesize...
    # Finally, return the row variable.
    return row

# This one's getting familiar. We are creating animations of reef images. In order to do so, we can create a function called create_animation along with the ims variable inside. But for reference, check out my notebook: EDA Solution Simplfified (make sure to like it!) https://www.kaggle.com/dinowun/eda-solution-simplified
def create_animation(ims):
    # We first set up our figure variable fig, to set up the plot with the plt module with the figure function which holds the figsize attribute to (16, 12).
    fig = plt.figure(figsize=(16, 12))
    # We can create a plot without axis by calling out the plt module with the axis function and set it to 'off'.
    plt.axis('off')
    # Now, we can create a variable called im and assign it to represent the data as an image by setting the plt module with linking the imshow function with the ims variable with the slice index of 0.
    im = plt.imshow(ims[0])
    
    # Time to create an animation function! We create a function called animate_func containing the i variable
    def animate_func(i):
        # We inherit the im variable to the animate_func function and plug the set_array function containing the ims variable of the create_animation function with the slice index of the i variable in animate_func.
        im.set_array(ims[0])
        # Finally, return the im variable enclosed on brackets.
        return [im]
    
    # Back inside the create_animation function, we finally return the animation module from matplotlib that is followed by the FuncAnimation module containing two variables: fig and animate_func (function call) and two attributes: frames set to the length (the len function) of the images variable, and the interval being set to 1000 divided by 12 (no remainders).
    return animation.FuncAnimation(fig, animate_func, frames=len(images), interval=1000//12)

# Now, after these functions, we set up the random seed by calling the np module with the random attribute along with the seed function and set it to 32.
np.random.seed(32)

# Finally, define a variable called colors and assign it to a tuple of four np modules with random attributes linking the randint function containing 255 along with the idx looping through a range function of 1 in a for loop after the \ function followed by a newline.
colors = [(np.random.randint(255), np.random.randint(255), np.random.randint(255), np.random.randint(255))\
          for idx in range(1)]

## Create BBox
After getting through the Helper code build, let's now make bboxes! We first redefine a variable called df and stick it with the bboxes key and assign it to itself with the annotations attribute and the progress_apply function containing the get_bbox function call so that it will append the 'bboxes' key to the df DataFrame. Finally, display out the df variable (DataFrame) with the head function calling out the first two rows.

In [ ]:
df['bboxes'] = df.annotations.progress_apply(get_bbox)
df.head(2)

## Obtain Image-Size
Time to measure the size of every image there! In order to get the image size of every image, we call out the df variable (or DataFrame) along with two keys, width and height and we'll assign the width key to 1280 and the height key to 720. Why? Because:
> All images have same dimension, [Width, Height] = `[1280, 720]`

After all, display the two head rows of the df DataFrame that has the annotations.

In [ ]:
df['width'] = 1280
df['height'] = 720
display(df.head(2))

## Create Labels
In order to create labels, We need to export our labels to convert it to **YOLO format**, with one `*.txt` file per every image (when if there are no object in image, there is no `*.txt` is required). The *.txt specifications are:
* One row every object
* Every row is in class of `[x_center, y_center, width, height]` format.
* Box coordinates must be in **normalized** `xywh` format (from 0 to 1). If our boxes are in pixels, we can divide `x_center` and `width` by `image width`, `y-center`, and `height` by `image_height`.

> The competition bbox format is exactly **COCO** hence the format of `[x_min, y_min, width, height]`. Without of doubt, we need to convert from **COCO** to **YOLO** format. 

With all of that let's demonstrate how to create labels with descriptions in the code cell below! (Unhide the code plz...)

In [ ]:
# First, we need to define a variable called cnt and set it to the value of 0.
cnt = 0
# We also define a variable called all_boxes and assign it to an empty list.
all_bboxes = []
# Now, we are going to create a for loop, defining a variable called row_idx and looping it to the tqdm function over the range (function) of df DataFrame with the shape attribute over a slice index of 0.
for row_idx in tqdm(range(df.shape[0])):
    # Inside this for loop, the row variable is assigned to the df DataFrame with the iloc to index through the row_idx variable.
    row = df.iloc[row_idx]
    # We define the image_height and the image_width and assign them to the row variable with the height and width attribute
    image_height = row.height
    image_width = row.width
    # Another variable we define is the bboxes_coco. We assign it to np module with the array module that represents an array of basic values to the row variable with the bboxes attribute followed by an astype function for a specific dtype of a np module with the float32 type and copy it with the copy function. 
    bboxes_coco = np.array(row.bboxes).astype(np.float32).copy()
    # We also define a variable called num_bbox and assign it to the length (len function) of the bboxes_coco variable.
    num_bbox = len(bboxes_coco)
    # Lastly for setup inside the for loop, we setup two variables: names and labels and assign them to the slice index of 'cots' (0 for labels) times the num_bbox variable.
    names = ['cots'] * num_bbox
    labels = [0] * num_bbox
    # Now, we are creating annotations (YOLO)! First of all, we are opening the file which is the the row variable with the label_path attribute and write it as the f variable.
    with open(row.label_path, 'w') as f:
        # After row.label_path file was opened, there was an if statement saying that the num_bbox variable is greater than 1. If the num_bbox greater than 1, then...
        if num_bbox < 1:
            # the annot variable was assigned to just an empty string.
            annot = ''
            # The file function, f, writes the annot string.
            f.write(annot)
            # Lastly, the cnt value variable will add 1 value.
            cnt += 1
            # And we have to continue outside the if statement.
            continue
        # Outside the for loop, the bboxes_yolo is defined as the coco2yolo function call with the height (image_height variable), width (image_width), and the bounding boxes in coco format (bboxes_coco). And it also defines the np module with the function, clip, that limits the values in an array withstanding three values, the bounding boxes in yolo format (bboxes_yolo variable), the number 0 and 1.
        bboxes_yolo = coco2yolo(image_height, image_width, bboxes_coco)
        bboxes_yolo = np.clip(bboxes_yolo, 0, 1)
        # We also call out the all_bboxes variable that has the empty array and extend it (with the extend function) containing the bounding boxes in yolo format variable, bboxes_yolo.
        all_bboxes.extend(bboxes_yolo)
        # And another for loop is made looping out the bbox_idx variable in the range (function) of the length (len function) of the bboxes_yolo (bounding boxes in yolo format) variable.
        for bbox_idx in range(len(bboxes_yolo)):
            # Inside the for loop, the annot variable is defined again, but this time, it is assigned to an array that contains the string (str function) of the labels variable with the slice index of bbox_idx variable plus the list (function) of bboxes_yolo (bbox format of yolo) variable with the slice index of bbox_idx variable followed by an astype function for a specific dtype of a str type plus the key of '\n' along with an if statement saying that if the num_bbox variable is not equal (!=) to the bbox_idx value variable plus 1 in parentheses thus otherwise its just an empty string key. It also redefined again to annexing (join function) itself to the empty space string and again to itself that stripped off the empty space of a string.
            annot = [str(labels[bbox_idx])] + list(bboxes_yolo[bbox_idx].astype(str)) + (['\n'] if num_bbox != (bbox_idx + 1) else [''])
            annot = ' '.join(annot)
            annot = annot.strip(' ')
            # Lastly, make the f variable write the annot string.
            f.write(annot)
            
# Now, let's print out how many missing bboxes are there!
print("BBoxes Missing:", cnt)

**Cool!** There are no bboxes missing after we created (and checked) annotation labels! After all, we are now proceeding to fold creation!

## Fold Creation
Before we create some folds, we know that the
> Number of samples aren't alike in every fold that can create huge variance in **Cross-Validation**.

Well, first of all, in order to create folds, we need to import the scikit-learn (sklearn) module with the model_selection section and import the GroupKFold function iterator variant from it. Then, we are defining variable kf to the GroupKFold iterator function variant, setting the n_splits attribute value to 5. Then we return new DataFrame by redefining the df DataFrame to itself with the reset_index with the drop attribute set to True. We then create a new key that is added to the df DataFrame that is called the fold key and we will set their value to -1. Next, we are creating a for loop, looping the fold variable and the tuple of train_idx and val_idx variable in add a counter with the enumerate function containing the kf GroupKFold iterator spliting out the df DataFrame, setting the y attribute to the df DataFrame with the video_id data converting to a list (tolist function), and setting the groups to the sequence of the df DataFrame. Inside the for loop, the df DataFrame with the loc attribute that accessed a group of rows and columns by labels of the val_idx variable and the 'fold' key is assigned to the fold variable. Lastly, display the value counts of the df DataFrame with the Fold key.

In [ ]:
from sklearn.model_selection import GroupKFold

kf = GroupKFold(n_splits=5)
df = df.reset_index(drop=True)
df['fold'] = -1

for fold, (train_idx, val_idx) in enumerate(kf.split(df, y=df.video_id.tolist(), groups=df.sequence)):
    df.loc[val_idx, 'fold'] = fold
display(df.fold.value_counts())

After running the results, we see that the name of the key in the df DataFrame is fold and the dtype of it is int64. Now let's make a dataset from the df!

## Dataset
In order to config a dataset, let's define two variables, train_files and val_files and assign it to an empty array. Then, we will define another two variables, train_df and valid_df and assign it to query the columns of a df DataFrame with a boolean expression of an if statement regarding whether the fold variable is (not for train_df variable) equal (for valid_df variable) to @FOLD thing. Now, the train_files and val_files variable added a value of a list of the train_df (valid_df) variable of image_path key into finding the unique elements of the image_path of the two variables mentioned (unique). Finally, print out the number of files in train_files and val_files variable.

In [ ]:
train_files = []
val_files = []

train_df = df.query("fold!=@FOLD")
valid_df = df.query("fold==@FOLD")

train_files += list(train_df.image_path.unique())
val_files += list(valid_df.image_path.unique())
len(train_files), len(val_files)

As you can see, there are 3974 files in the train_files section, and there are 945 files in the val_files section. With all being set, let's now config the dataset config file!

## Configuring the Dataset Config File
In order to config the dataset config file, it requires:
1. The dataset root directory path and the other relative paths leading to `train/val/test` image directories (or rather, *.txt files that contain image paths...)
2. The no. of classes `nc` and...
3. A list of class `names`: `['cots']`

First of all, We have to import the yaml module so that we will save the config file in a yolo format. Then, set the cwd variable to the path in a string, kaggle/working. Then, open the file joined by the os module followed by the path attribute with a join function along with the train(val).txt file and write it as the f variable. When writing the train.txt or val.txt file, there is a for loop looping the path variable in the the valid_df DataFrame with the image_path key that is converted to list by the tolist function. Then, define another variable, data to the dictionary creation using the dict function, so that it would be easy to assign 5 variables inside: set path to kaggle/working, in strings, train (or val) to joining the path of cwd variable and a train(val).txt using os (the module with the path attribute followed by the join function), nc set to 1, and names set to the cots key. We now open the file path that the os module combined (with the path attribute followed by a join function) of the cwd variable path and the bgr.yaml file thus writing it as the outfile variable. When writing the path provided by cwd variable to the bgr.yaml file, the yaml module serialized a Python object into a YAML stream containing the data dictionary variable, the outfile variable that is writing the bgr.yaml, and set the default_flow_style that the HTTP URL to send the request to the yaml file has set to False. Finally, set the f variable that writes the train and val txt file to open the full file between the cwd variable path and the bgr.yaml supported by the os module that has the path attribute with the join function). We then print out the newline character with the yaml in a string and the f variable file being read.

In [ ]:
import yaml

cwd = '/kaggle/working/'

with open(os.path.join(cwd, 'train.txt'), 'w') as f:
    for path in train_df.image_path.tolist():
        f.write(path + '\n')
        
with open(os.path.join(cwd, 'val.txt'), 'w') as f:
    for path in train_df.image_path.tolist():
        f.write(path + '\n')
        
data = dict(
    path = '/kaggle/working',
    train = os.path.join(cwd, 'train.txt'),
    val = os.path.join(cwd, 'val.txt'),
    nc = 1,
    names = ['cots'],
)

with open(os.path.join(cwd, 'bgr.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style = False)
    
f = open(os.path.join(cwd, 'bgr.yaml'), 'r')
print("yaml:")
print(f.read())

When reading the bgr.yaml file provided by the f variable file, then the names would be just cots key, the nc is set to 1, the path was set to kaggle working, the train has set to the path leading to train.txt, and the val has set to the the same thing as train but with the val.txt file.

## [YOLOv5](https://github.com/ultralytics/yolov5/)
![Here we go! YOLOv5!](https://github.com/ultralytics/yolov5/releases/download/v1.0/splash.jpg)
This is the part we all been waiting for, the YOLOv5! It is the most valuable object detection algorithm that can detect the infamous cots without any flaws. So Let's start the fun of the YOLO(v5)!

### Installation
First before going to YOLOv5, we change the directory path (cd) to /kaggle/working/, then remove the yolov5 folder in the kaggle/working directory (if possible). Later on, we clone out the yolov5 repository via git clone command (https://github.com/ultralytics/yolov5). Next, we copy the yolov5 lib ds (the dataset you may get in kaggle) to the yolov5 directory. Finally, change directory to yolov5 and install the requirements in the txt file via pip. To get started on YOLOv5, we first import utils from their module and then check YOLOv5 by assigning the display variable to the utils module with the notebook_init function.

In [ ]:
%cd /kaggle/working
!rm -r /kaggle/working/yolov5 # if possible
!git clone https://github.com/ultralytics/yolov5 # clone it!
!cp -r /kaggle/input/yolov5-lib-ds /kaggle/working/yolov5
%cd yolov5
%pip install -qr requirements.txt # install it!

from yolov5 import utils
display = utils.notebook_init() # check it!

And we have done started setting up YOLOv5 without failure! Without further ado, let's train YOLOv5 correctly by setting up the custom hyperparameters!

### Custom Hyperparameters (Fine-Tuning)
Now, let's get some training over detecting COTS! In order to fine-tune Yolov5 well, we reimport the os and yaml module again, then setup the cwd variable to the file path leading to custom.yaml file. We then set up the data variable followed by a hyperparams variable to the dictionary containing lr0, lrf, momentum, weight_decay, warmup_epochs, warmup_momentum, warmup_bias_lr, box, cls, obj, obj_pw, iou_t, anchor_t, fl_gamma, hsv_h, hsv_s, hsv_v, degrees, translate, scale, shear, perspective, flipud, fliplr, mosaic, mixup, and copy_paste to any value that can make our model detect cots better. Like configuring the dataset config file, we open the file provided by the cwd variable and write it as the outfile variable. When writing the cwd variable file, the yaml module serialized a Python object into a YAML stream containing the hyperparams variable with the dictionary in the data variable, the outfile variable that is writing the bgr.yaml, and set the default_flow_style that the HTTP URL to send the request to the yaml file has set to False. Finally, set the f variable to open the cwd file variable and set it to read mode and print out the yaml file being read by the f variable (although you have to print out "yaml:" in strings).

In [ ]:
import os
import yaml

cwd = '/kaggle/working/custom.yaml'

# Change it whatever you want!
data = hyperparams = {
    'lr0': 0.01,
    'lrf': 0.1,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 5.0,
    'warmup_momentum': 0.8,
    'warmup_bias_lr': 0.1,
    'box': 0.05,
    'cls': 0.5,
    'cls_pw': 1.0,
    'obj': 1.0,
    'obj_pw': 1.0,
    'iou_t': 0.2,
    'anchor_t': 4.0,
    'fl_gamma': 0.0,
    'hsv_h': 0.015,
    'hsv_s': 0.7,
    'hsv_v': 0.3,
    'degrees': 0.0,
    'translate': 0.1,
    'scale': 0.7,
    'shear': 0.0,
    'perspective': 0.0,
    'flipud': 0.0,
    'fliplr': 0.5,
    'mosaic': 0.5, #0.0 or 1.0 would be better and recommended
    'mixup': 0.5,
    'copy_paste': 0.0
}

with open(os.path.join(cwd), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)
    
f = open(os.path.join(cwd), 'r')
print('yaml:')
print(f.read())

### Weights and Biases (Optional)
Like mentioned, Weights & Biases is useful for detecting performance of object detection. So, you'll mind that this is optional. All you have to do is to import wandb and then login to it by anonymous.

In [ ]:
import wandb
wandb.login(anonymous='must')

## Training
Now've setup our object-detection model with YOLOv5, let's now train it! We run the train.py file using python3 command and set the hyperparameters to the path leading towards custom.yaml file, set the batch to 12 (set it whatever you want...), epochs to 18 (or more of that, maybe 20), data to the path leading toward the bgr.yaml file, weights set to yolov5m.pt file, and set no workers.

In [ ]:
# We should clean our memory first...
import gc
gc.collect()

!python3 train.py --img 1280\
--hyp /kaggle/working/custom.yaml\
--batch 12\
--epochs 20\
--data /kaggle/working/bgr.yaml\
--weights yolov5m.pt --workers 0

## Output Files
After running through all of the painful epoch training, let's take a peek at the output files! We run the ls command to take a peek at the runs/train/exp directory.

In [ ]:
!ls runs/train/exp

## Class Distribution
Since we took a peek at the runs/train, let's take some peek at the class distribution! We look at the cots labels in a correlogram graph image file. First we import the matplotlib with pyplot module again as plt, set the graph figure size width and height to 10 units. Then, disable the axis using plt module with the axis function and set the string 'off'. Finally, display the graph of the correlogram of labels being read out from the imread function over the path to the labels_correlogram.jpg file.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,10))
plt.axis('off')
plt.imshow(plt.imread('runs/train/exp/labels_correlogram.jpg'))
plt.show()

Now we will see our labels analyzed seperately in a graph. Its just like what we have looked in the labels in a correlogram but this time, it displayed the graph of the four seperate label graphs being read out from the imread function over the path to the labels.jpg file.

In [ ]:
plt.figure(figsize=(10,10))
plt.axis('off')
plt.imshow(plt.imread('runs/train/exp/labels.jpg'))
plt.show()

## Batch Image
To get the batch images plotted, we create a figure whose width and height size is 10 three times, and show the image read out by the plt module with the imread function to the path leading to train_batch0 to train_batch2.jpg file.

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(plt.imread('runs/train/exp/train_batch0.jpg'))

plt.figure(figsize=(10, 10))
plt.imshow(plt.imread('runs/train/exp/train_batch1.jpg'))

plt.figure(figsize=(10, 10))
plt.imshow(plt.imread('runs/train/exp/train_batch2.jpg'))

## GT vs Predictions
After running the batch images, let's now compare GT and our predictions! In order to compare our predictions with the GT, we assign two variables: fig and ax to the subplots function that creates a figure with a set of subplots already made with the numbers 3, 2, setting the width and height to 18 units and 15 units, and the constrained_layout attribute set to True for every seperate image graphs. We later then set a for loop looping the row variable to the range of 3. Inside there, we call out the ax variable with the row variable in the index along with the 0 and 1 index seperate to show the path towards the val_batch with values from 0 to 2 in the row variable over the labels(pred).jpg file, then setting the x and y ticks, and finally set title over the file path leading to val_batch with values from 0 to 2 in the row vairable over the labels(pred).jpg file, although we have to set the font size to 12. Now, we show the image graph.

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(2 * 9 ,3 * 5), constrained_layout=True)

for row in range(3):
    ax[row][0].imshow(plt.imread(f'runs/train/exp/val_batch{row}_labels.jpg'))
    ax[row][0].set_xticks([])
    ax[row][0].set_yticks([])
    ax[row][0].set_title(f'runs/train/exp/val_batch{row}_labels.jpg', fontsize=12)
    
    ax[row][1].imshow(plt.imread(f'runs/train/exp/val_batch{row}_pred.jpg'))
    ax[row][1].set_xticks([])
    ax[row][1].set_yticks([])
    ax[row][1].set_title(f'runs/train/exp/val_batch{row}_pred.jpg', fontsize=12)

plt.show()

## The Results
After training every epoch in our object detection model, Let's get started over graphing over the results with every epoch and score in matplot style!

### Score vs Epoch
In order to see the score being compared to an epoch, we set the figure width and height to 30 and 15 units, turn off the axis using the axis function, and then show the path to results.png file being read out. Finally, show the image graph.

In [ ]:
plt.figure(figsize=(30,15))
plt.axis('off')
plt.imshow(plt.imread('runs/train/exp/results.png'))
plt.show()

### Confusion Matrix
Now we create a confusion matrix based on our predicted labels over COTS! We first set up the width and height figure size to 12 and 10 units, turn off the axis using the axis function again, then show the path to results.png file being read out. Finally, show the image graph using show function.

In [ ]:
plt.figure(figsize=(12,10))
plt.axis('off')
plt.imshow(plt.imread('runs/train/exp/confusion_matrix.png'))
plt.show()

### Metrics
Now, lets finally display more results, in metrics! We first set up a for loop, looping the metric variable to an array of four strings, 'F1', 'PR', 'P', and 'R'. Inside this for loop, the metric type will be printed out, then the figure's width and height has been set up to 12 and 10 units, thus turning off the axis using the axis function. Next, the image graph is shown by the file path to any metric looped by the metric variable over the curve.png file being read out. Finally, show the image file using the show function.

In [ ]:
for metric in ['F1', 'PR', 'P', 'R']:
    print(f"Metric: {metric}")
    plt.figure(figsize=(12, 10))
    plt.axis('off')
    plt.imshow(plt.imread(f'runs/train/exp/{metric}_curve.png'));
    plt.show()

## Conclusion
After running through all of the training over object detection, we are pretty sure that we have done the training part for our YOLOv5 Simplified Series! But for sure, we are going to create another notebook about infering YOLOv5, so stay tuned!

## Acknoledgements and References
https://www.kaggle.com/csarolivares/own-great-barrier-reef-yolov5-train/notebook
https://www.kaggle.com/awsaf49/great-barrier-reef-yolov5-train